In [1]:
import joblib
import re
import jieba
import zhconv
import os

In [2]:
def tr_to_sim(text):
    res = zhconv.convert(text, "zh-cn")
    return res

def jieba_cut_text(text, stop_word, cut_by=" "):

    text = tr_to_sim(text)
    # [\u4E00-\u9FA5]+ 去除非中文字符
    patt = re.compile("[\u4E00-\u9FA5]+", re.VERBOSE|re.IGNORECASE)
    word_list = []
    for t in patt.findall(text):
        word_list += [word for word in jieba.cut(t) if word not in stop_word]
    return cut_by.join(word_list)

In [3]:
path = r".\flask_online\model"
le = joblib.load(os.path.join(path, "labelr.pkl"))
tfidf = joblib.load(os.path.join(path, "tfidf.pkl"))
clf = joblib.load(os.path.join(path, "logi.pkl"))

In [10]:
with open(r"stopwords.txt", 'r', encoding="utf-8") as fp:
    stopw = fp.read().split(" ")
stopw[: 20]

['!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '--',
 '.',
 '..',
 '...',
 '......',
 '...................',
 './']

In [12]:
s = jieba_cut_text("你这个傻逼", stopw)
s

'傻 逼'

In [14]:
v = tfidf.transform([s])
v

<1x110923 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in Compressed Sparse Row format>

In [15]:
clf.predict(v)

array([1])

In [16]:
le.classes_

array(['正向', '负向'], dtype=object)

In [17]:
le.inverse_transform(clf.predict(v))

array(['负向'], dtype=object)